In [1]:
import os
import requests
import json
import numpy as np
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

from pymongo import MongoClient
import time

import foursquare as fs

In [2]:
client = MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("companies")

In [3]:
def getCompanies_TagCityCountry(tag, country, city):
    filter_ = {'tag_list':{'$regex':tag}, 'offices.city':city, 'offices.country_code':country}
    projection = {'name':1, '_id':0}
    result = list(c.find(filter_, projection))
    return result

In [4]:
def getCompaniesAndLocation(category):
    filter_ = {"category_code":category}
    projection = {'category_code':1, 'name':1, 'offices.city':1, 'offices.country_code':1, 'offices.latitude':1, 'offices.longitude':1, '_id':0}
    result = list(c.find(filter_, projection))
    return result

In [5]:
def unflatten_result(result):
    companies = []
    for company in result:
        aux = []
        name = company['name']
        location = [value for element in list(company['offices']) for key,value in element.items()]
        location.insert(0,name)
        companies.append(location[:5])
    return companies

In [6]:
def getTopCountry(base_df):
    countriesOrdered = base_df.groupby('Country').count()
    return countriesOrdered.sort_values(by=['Company'], ascending=False).index[0]

In [7]:
def getTop3Cities(base_df, country):
    df_topCountry = base_df.loc[base_df['Country']==country]
    df_topCountry = df_topCountry.sort_values(by=['City']).replace(to_replace='None', value=np.nan).dropna()
    return list(df_topCountry.groupby('City').count().sort_values(by=['Company'], ascending=False).index[:3])   

In [8]:
def buildTopCitiesDF(base_df, country, cities):
    df = base_df.loc[(base_df['Country']==country) &
                 ((base_df['City']==cities[0]) | (base_df['City']==cities[1]) | (base_df['City']==cities[2]))]
    return df.sort_values(by=['City'])